In [1]:
import pandas as pd
import datetime
from sklearn import datasets

In [2]:
from evidently.ui.workspace.cloud import CloudWorkspace

from evidently.report import Report
from evidently.metric_preset import DataQualityPreset
from evidently.metric_preset import DataDriftPreset
from evidently.metrics import *
from evidently.test_suite import TestSuite
from evidently.tests import *
from evidently.test_preset import DataDriftTestPreset
from evidently.tests.base_test import TestResult, TestStatus

In [3]:
adult_data = datasets.fetch_openml(name="adult", version=2, as_frame="auto")
adult = adult_data.frame
adult_ref = adult[~adult.education.isin(["Some-college", "HS-grad", "Bachelors"])]
adult_prod = adult[adult.education.isin(["Some-college", "HS-grad", "Bachelors"])]

/Users/sakshamgulati/.local/share/virtualenvs/MLOps_Template-k_6XISXV/lib/python3.10/site-packages/sklearn/datasets/_openml.py:1022: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [20]:
from dotenv import load_dotenv
import os

load_dotenv()
#connecting to evidently cloud workspace using API token
ws = CloudWorkspace(
token=os.getenv("evi_api"),
url="https://app.evidently.cloud")

# #creating a project in the workspace
__project_name__="Test Project"
__team_id__="74fdd884-9679-4693-819b-a6695e652e25"

project = ws.create_project(__project_name__,team_id=__team_id__)
project.description = "My project description"
project.save()

Project(id=UUID('8ba8c5df-479c-491b-8dbd-d00192e048ed'), name='Test project', description='My project description', dashboard=DashboardConfig(name='Test project', panels=[], tabs=[], tab_id_to_panel_ids={}), team_id=UUID('74fdd884-9679-4693-819b-a6695e652e25'), date_from=None, date_to=None, created_at=datetime.datetime(2024, 6, 10, 21, 40, 9, 847782))

In [13]:
project = ws.create_project("My project name", team_id="74fdd884-9679-4693-819b-a6695e652e25")

InvalidHeader: Header part (None) from ('X-Evidently-Token', None) must be of type str or bytes, not <class 'NoneType'>

In [ ]:
data_report = Report(
        metrics=[
            DataDriftPreset(stattest='psi', stattest_threshold='0.3'),
            DataQualityPreset(),
        ],
        timestamp=datetime.datetime.now(),
    )

data_report.run(reference_data=adult_ref, current_data=adult_prod.iloc[0 : 100, :])